In [39]:
"""
Assignment 4
1.	Import training data provided in Assignment_4_Data_and_Template.xlsx 
2.	Use the pseudoinverse method on the given training data to build two linear classifiers: (i) 
a binary classifier, and (ii) a multiclass classifier capable of distinguishing between 6 classes. 
The sheet titled "Training Data" contains the target class labels (Failure and Type). 
Paste your classifiers in the correct format as shown in the sheet titled "Classifiers"
3.	Apply the classifiers to the data provided in the sheet "To be classified." 
Paste the results into the sheet.
4.	Evaluate the performance of your classifiers using the metrics requested in the sheet "Performance"
For guidance, see slides titled "Step-by-step: Linear Classifier Design" in the lecture on linear 
classifiers.
A completed spreadsheet is your submission. Except for writing results into the correct cells, 
do not alter the spreadsheets in any other way.
 
Extra Credit Opportunity: 
Flex your linear algebra muscles, mindfully!
Refer to the slide titled "What the pseudoinverse does"
•	In the context of Assignment 4, what does  wbest represent?
•	In the context of Assignment 4, what does  b represent?
•	Can you compute the vector  bp of Assignment 4?
•	Can you compute 5 random vectors ( b1,b2,b3,b4,b5) lying on the "Assignment 4 counterpart of the shaded 
plane" in the slide?
•	Using distance calculations, can you check that none of these random vectors are as close 
to  b as  bp is?
Use Assignment_4_Extra_Credit.xlsx  to submit your answers. Good luck and enjoy!
NOTE: Submitting a correct solution to this assignment will potentially get you a score of 125/100 for 
Assignment 4. [Similar to the situation in Assignment 2.] Please write to me if you need further 
clarification.

"""

'\nAssignment 4\n1.\tImport training data provided in Assignment_4_Data_and_Template.xlsx \n2.\tUse the pseudoinverse method on the given training data to build two linear classifiers: (i) \na binary classifier, and (ii) a multiclass classifier capable of distinguishing between 6 classes. \nThe sheet titled "Training Data" contains the target class labels (Failure and Type). \nPaste your classifiers in the correct format as shown in the sheet titled "Classifiers"\n3.\tApply the classifiers to the data provided in the sheet "To be classified." \nPaste the results into the sheet.\n4.\tEvaluate the performance of your classifiers using the metrics requested in the sheet "Performance"\nFor guidance, see slides titled "Step-by-step: Linear Classifier Design" in the lecture on linear \nclassifiers.\nA completed spreadsheet is your submission. Except for writing results into the correct cells, \ndo not alter the spreadsheets in any other way.\n \nExtra Credit Opportunity: \nFlex your linear a

In [66]:
import numpy as np
from matplotlib import pyplot as plt

def getSheetNames(excelfile):
    from pandas import ExcelFile
    return (ExcelFile(excelfile)).sheet_names

def readExcelSheet1(excelfile):
    from pandas import read_excel
    return (read_excel(excelfile)).values

def readExcelRange(excelfile, sheetname="Sheet1", startrow=1, endrow=1, startcol=1, endcol=1):
    from pandas import read_excel
    values = (read_excel(excelfile, sheetname, header=None)).values;
    return values[startrow - 1:endrow, startcol - 1:endcol]

def readExcel(excelfile, **args):
    if args:
        data = readExcelRange(excelfile, **args)
    else:
        data = readExcelSheet1(excelfile)
    if data.shape == (1, 1):
        return data[0, 0]
    elif (data.shape)[0] == 1:
        return data[0]
    else:
        return data
def writeExcelData(x, excelfile, sheetname, startrow, startcol, reshape=None):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    if reshape:
        df = DataFrame(x.reshape(-1, len(x)))
    else:
        df = DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname, startrow=startrow - 1, startcol=startcol - 1, header=False, index=False)
    writer.save()
    writer.close()


excelfile = r"/Users/omerorhan/Desktop/UCSC/MachineLearning/Assignments/venv/notebookfiles/" \
            r"Assignment#4/Assignment_4_Data_and_Template.xlsx";
sheets = getSheetNames(excelfile)
sheets

data = readExcel(excelfile)



In [65]:
def getClassifier(data):
#Collect and assemble 𝑁×𝑑 matrix of feature vectors into matrix 𝐗. 
    X = data[:,:-4]
    #Assemble targets into 𝐓
    F = np.array(data[:,-4:-3], dtype=int)
    Tnum =  np.array(data[:,-3:-2], dtype=int)
    N = len(X)
    w = np.ones(N)
    w[0] = 0 
    ones = np.ones((N,1),dtype=int)
    #Construct the augmented feature vector set 𝐗𝐚 by prepending a column of 1s in front of the matrix 𝐗.
    # Note that 𝐗𝐚 has dimensions 𝑁×(𝑑+1).
    Xa = np.concatenate((ones,X), axis=1)
    print(Xa.shape)
    XaPI = np.linalg.pinv(Xa.astype(int))
    wF= np.matmul(XaPI,F)
    wT = np.matmul(XaPI, Tnum)
    return wF, wT, X, F, Tnum, N

